In [228]:
import pandas as pd
import pyreadstat 
import chardet
import numpy as np

In [331]:
#Import the csv
population = pd.read_csv('./Datasets/Populationbyregion.csv').drop_duplicates()
#Get rid of some of the forests that sit in different regions by adding them together
totals=population.groupby(['county','indicator'])['Value'].sum().reset_index()
#Pivot to get a wide rather than a narrow dataset
population_data = totals.pivot(index=['county'],columns='indicator', values='Value')
#The household size data is currently one significant figure, recalculate
population_data['Average Household size'] = population_data['Population, Total']/population_data['Number of Households']
population_data=population_data.reset_index()

In [151]:
mtf_battery_df, mtf_battery_meta = pyreadstat.read_dta('./Datasets/Kenya_MTF_data/MTF_HH_Battery_Data_Final.dta')
mtf_weight_df, mtf_weight_meta = pyreadstat.read_dta('./Datasets/Kenya_MTF_data/weight.dta')

In [152]:
with_weights = pd.merge(mtf_battery_df, mtf_weight_df[['PARENT_KEY','pw_final']], on = 'PARENT_KEY')
max_vals=with_weights.groupby(['PARENT_KEY'])['pw_final', 'c_multiple'].max().reset_index()
sum_vals=with_weights.groupby(['PARENT_KEY'])['c_c_110', 'c_c_111'].sum().reset_index()
battery_hhs = pd.merge(max_vals,sum_vals,on = ['PARENT_KEY']).rename(columns={'c_c_110':'total_battery_capacity', 'c_c_111':'total_battery_voltage','c_multiple':'num_batteries'})

/var/folders/wh/kn94yryj1b35n6692fmfv3dc0000gq/T/ipykernel_67333/1801959063.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  max_vals=with_weights.groupby(['PARENT_KEY'])['pw_final', 'c_multiple'].max().reset_index()
/var/folders/wh/kn94yryj1b35n6692fmfv3dc0000gq/T/ipykernel_67333/1801959063.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  sum_vals=with_weights.groupby(['PARENT_KEY'])['c_c_110', 'c_c_111'].sum().reset_index()


In [229]:
mtf_core_df = pd.read_stata('./Datasets/Kenya_MTF_data/MTF_HH_Core_survey_Final_data_trimmed-2.dta', convert_categoricals=False).drop(columns=['enum_id', 'enum_foot','enum_ht','sup_id','tab_id','subdate','start_time'])
with_weights_survey = pd.merge(mtf_core_df, mtf_weight_df[['PARENT_KEY','pw_final']], on = 'PARENT_KEY')
max_vals_survey=with_weights_survey.groupby(['PARENT_KEY','HH_code','prov','cty','dist','div','loc', 'subloc','class', 'locality_ur', 'marg', 'grid_loc'])['pw_final', 'a_a_1','b_b_6a','b_b_7'].max().reset_index().rename(columns={'a_a_1':'household_size','b_b_6a':'num_hhd_in_dwelling','b_b_7':'own_dwelling'})
mean_vals_survey=with_weights_survey.groupby(['PARENT_KEY','HH_code','prov','cty','dist','div','loc', 'subloc','class', 'locality_ur', 'marg', 'grid_loc'])['b_b_9','c_c_3','c_c_21','c_c_25ai','c_c_25aii'].mean().reset_index().rename(columns={'b_b_9':'rooms_occupied','c_c_3':'distance_to_grid','c_c_21':'avg_electricity_consumption_kwh','c_c_25ai':'worst_hours_electricity','c_c_25aii':'typical_hours_electricity'})
min_vals_survey=with_weights_survey.groupby(['PARENT_KEY','HH_code','prov','cty','dist','div','loc', 'subloc','class', 'locality_ur', 'marg', 'grid_loc'])[ 'b_b_16','b_b_21','c_c_2','c_c_38','c_c_75','c_c_104'].min().reset_index().rename(columns={'b_b_16':'bank_account_held','b_b_21':'mobile_money_acc_held','c_c_2':'grid_connection','c_c_38':'mini_grid_connection','c_c_75':'generator_for_electricity','c_c_104':'rechargeable_battery_electricity'})
total_survey = pd.merge(pd.merge(max_vals_survey, mean_vals_survey, on = ['PARENT_KEY','HH_code','prov','cty','dist','div','loc', 'subloc','class', 'locality_ur', 'marg', 'grid_loc']), min_vals_survey, on = ['PARENT_KEY','HH_code','prov','cty','dist','div','loc', 'subloc','class', 'locality_ur', 'marg', 'grid_loc'])

/Users/rebecca.duke/miniforge3/lib/python3.9/site-packages/pandas/io/stata.py:1514: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  warnings.warn(msg, UnicodeWarning)
/var/folders/wh/kn94yryj1b35n6692fmfv3dc0000gq/T/ipykernel_67333/1727438286.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  max_vals_survey=with_weights_survey.groupby(['PARENT_KEY','HH_code','prov','cty','dist','div','loc', 'subloc','class', 'locality_ur', 'marg', 'grid_loc'])['pw_final', 'a_a_1','b_b_6a','b_b_7'].max().reset_index().rename(columns={'a_a_1':'household_size','b_b_6a':'num_hhd_in_dwelling','b_b_7':'own_dwelling'})
/var/folders/wh/kn94yryj1b35n6692fmfv3dc0000gq/T/ipykernel_67333/1727

In [230]:
mtf_solar_df = pd.read_stata('./Datasets/Kenya_MTF_data/MTF_HH_Solar_Roster.dta')
with_weights_solar = pd.merge(mtf_solar_df, mtf_weight_df[['PARENT_KEY','pw_final']], on = 'PARENT_KEY')
max_vals_solar=with_weights_solar.groupby(['PARENT_KEY'])['pw_final', 'c_position'].max().reset_index()
sum_vals_solar=with_weights_solar.groupby(['PARENT_KEY'])['c_131_pwr','c_133_batt'].sum().reset_index()
solar_hhs = pd.merge(max_vals_solar,sum_vals_solar,on = ['PARENT_KEY']).rename(columns={'c_133_batt':'solar_battery_capacity', 'c_131_pwr':'total_solar_rating','c_position':'num_solar'})

/var/folders/wh/kn94yryj1b35n6692fmfv3dc0000gq/T/ipykernel_67333/2366092681.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  max_vals_solar=with_weights_solar.groupby(['PARENT_KEY'])['pw_final', 'c_position'].max().reset_index()
/var/folders/wh/kn94yryj1b35n6692fmfv3dc0000gq/T/ipykernel_67333/2366092681.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  sum_vals_solar=with_weights_solar.groupby(['PARENT_KEY'])['c_131_pwr','c_133_batt'].sum().reset_index()


In [295]:
raw_mtf_data =pd.merge(total_survey,pd.merge(battery_hhs,solar_hhs, how='outer',on='PARENT_KEY').drop(columns=(['pw_final_x','pw_final_y'])), on='PARENT_KEY', how='left')



In [310]:
raw_mtf_data['weightedhousehold_inhabitants'] = raw_mtf_data.pw_final*raw_mtf_data.household_size
raw_mtf_data.loc[np.isnan(raw_mtf_data.num_hhd_in_dwelling),'num_hhd_in_dwelling'] = 0
raw_mtf_data['weighted_hh_per_dwelling'] = raw_mtf_data.pw_final*(raw_mtf_data.num_hhd_in_dwelling+1)
raw_mtf_data.loc[raw_mtf_data.own_dwelling==2,'own_dwelling'] = 0
raw_mtf_data['weighted_owned_dwelling'] = raw_mtf_data.pw_final*raw_mtf_data.own_dwelling
raw_mtf_data['weighted_rooms_occupied'] = raw_mtf_data.pw_final*raw_mtf_data.rooms_occupied
raw_mtf_data.loc[raw_mtf_data.grid_connection==2,'grid_connection'] = 0
raw_mtf_data['weighted_grid_connection'] = raw_mtf_data.pw_final*raw_mtf_data.grid_connection
raw_mtf_data.loc[raw_mtf_data.bank_account_held==2,'bank_account_held'] = 0
raw_mtf_data['weighted_bank_account'] = raw_mtf_data.pw_final*raw_mtf_data.bank_account_held
raw_mtf_data.loc[raw_mtf_data.mobile_money_acc_held==2,'mobile_money_acc_held'] = 0
raw_mtf_data['weighted_mobile_money'] = raw_mtf_data.pw_final*raw_mtf_data.mobile_money_acc_held
raw_mtf_data.loc[np.isnan(raw_mtf_data.num_batteries),'num_batteries'] = 0
raw_mtf_data['batteries_flag'] = 1
raw_mtf_data.loc[raw_mtf_data.num_batteries==0,'batteries_flag']=0
raw_mtf_data['weighted_batteries'] = raw_mtf_data.pw_final*raw_mtf_data.num_batteries
raw_mtf_data['weighted_battery_flag'] = raw_mtf_data.pw_final*raw_mtf_data.batteries_flag
raw_mtf_data.loc[np.isnan(raw_mtf_data.num_solar),'num_solar'] = 0
raw_mtf_data['solar_flag'] = 1
raw_mtf_data.loc[raw_mtf_data.num_solar==0,'solar_flag']=0
raw_mtf_data['weighted_solar'] = raw_mtf_data.pw_final*raw_mtf_data.num_solar
raw_mtf_data['weighted_solar_flag'] = raw_mtf_data.pw_final*raw_mtf_data.solar_flag
raw_mtf_data.loc[raw_mtf_data.generator_for_electricity==2,'generator_for_electricity'] = 0
raw_mtf_data['weighted_generator'] = raw_mtf_data.pw_final*raw_mtf_data.generator_for_electricity


Need to tidy up the categories so they agree with the level 1 administrative regions

In [425]:
raw_mtf_data.loc[raw_mtf_data.dist == 'Trans Nzoia West','dist'] = 'Trans Nzoia'
raw_mtf_data.loc[raw_mtf_data.dist == 'Taita','dist'] = 'Taita Taveta'
raw_mtf_data.loc[raw_mtf_data.dist == 'Mandera Central','dist'] = 'Mandera'
raw_mtf_data.loc[raw_mtf_data.dist.isin(['Nairobi West','Nairobi East','Nairobi North']),'dist'] = 'Nairobi'
raw_mtf_data.loc[raw_mtf_data.dist.isin(['Nyandarua North','Nyandarua South']),'dist'] = 'Nyandarua'
raw_mtf_data.loc[raw_mtf_data.dist.isin(['Kakamega Central','Kakamega North']),'dist'] = 'Kakamega'
raw_mtf_data.loc[raw_mtf_data.dist.isin(['Kisii Central','Kisii South']),'dist'] = 'Kisii'
raw_mtf_data.loc[raw_mtf_data.dist.isin(['Nandi Central','Nandi East','Nandi North']),'dist'] = 'Nandi'
raw_mtf_data.loc[raw_mtf_data.dist.isin(['Meru Central']),'dist'] = 'Meru'
raw_mtf_data.loc[raw_mtf_data.dist.isin(['Bungoma South','Bungoma West','Bungoma North']),'dist'] = 'Bungoma'
raw_mtf_data.loc[raw_mtf_data.dist.isin(['Muranga North','Muranga South']),'dist'] = "Murang'a"
raw_mtf_data.loc[raw_mtf_data.dist.isin(['Nyeri North','Nyeri South']),'dist'] = "Nyeri"
raw_mtf_data.loc[raw_mtf_data.dist.isin(['Homabay']),'dist'] = 'Homa Bay'
raw_mtf_data.loc[raw_mtf_data.dist.isin(['Samburu Central']),'dist'] = 'Samburu'


In [430]:
import geopandas
level2_boundary = geopandas.read_file("./Datasets/Shape files/ken_adm_iebc_20191031_shp/ken_admbnda_adm2_iebc_20191031.shp")
level2_boundary_lookup = level2_boundary[['ADM1_EN','ADM2_EN']]
level1_info = level2_boundary.ADM1_EN.drop_duplicates()
match_div2 = pd.merge(raw_mtf_data[['prov','cty','dist','div','loc']], level2_boundary_lookup, left_on='dist', right_on='ADM2_EN', how = 'left')
match_div2.loc[match_div2.dist.isin(list(level1_info)),'ADM1_EN']=match_div2.loc[match_div2.dist.isin(list(level1_info)),'dist']
match_div2.dist= match_div2.dist.str.upper()
district_county_lookup=pd.read_csv('District_to_county_lookup.csv').drop(columns='Unnamed: 0')

In [435]:
match_div2

,prov,cty,dist,div,loc,ADM1_EN,ADM2_EN
0,North Eastern,21.0,GARISSA,Central,Central,Garissa,NaN
1,Central,6.0,THIKA WEST,Thika West,Makongeni,NaN,NaN
2,Rift Valley,33.0,TRANS NZOIA,Kiminini,Kiminini,Trans Nzoia,NaN
3,Rift Valley,43.0,BOMET,Sigor,Sigor,Bomet,NaN
4,Rift Valley,30.0,TURKANA SOUTH,Katilu,Katilu,Turkana,Turkana South
...,...,...,...,...,...,...,...
4581,Coast,9.0,KILIFI,Kikambala,Junju,Kilifi,NaN
4582,Western,44.0,LUGARI,Likuyani,Nzoia,Kakamega,Lugari
4583,Coast,8.0,MSAMBWENI,Lunga Lunga,Lunga Lunga,Kwale,Msambweni
4584,Eastern,20.0,NZAUI,Mulala,Emali,NaN,NaN


In [467]:
with_district_lookup = pd.merge(match_div2,district_county_lookup,left_on='dist', right_on = 'District')
with_district_lookup.loc[pd.isna(with_district_lookup.ADM1_EN_x),'ADM1_EN_x'] = with_district_lookup.loc[pd.isna(with_district_lookup.ADM1_EN_x),'ADM1_EN_y']
with_district_lookup.ADM1_EN_x= with_district_lookup.ADM1_EN_x.str.upper()

In [468]:
raw_mtf_data.dist=raw_mtf_data.dist.str.upper()
raw_mtf_data_new=pd.merge(raw_mtf_data,with_district_lookup.drop(columns=['prov','cty','div','loc','ADM1_EN_y','ADM2_EN','District']),on='dist').rename(columns={'ADM1_EN_x':'county'})

In [471]:
raw_mtf_data_new.county.sort_values().drop_duplicates()

317618            BARINGO
7971                BOMET
309050            BUNGOMA
331652              BUSIA
256997    ELGEYO-MARAKWET
327594               EMBU
0                 GARISSA
318569           HOMA BAY
264264             ISIOLO
331355           KAKAMEGA
273970            KERICHO
275384             KIAMBU
40886              KILIFI
300170          KIRINYAGA
273439              KISII
306117             KISUMU
276608              KITUI
328845              KWALE
313665               LAMU
24027            MACHAKOS
315045            MAKUENI
262477            MANDERA
330804           MARSABIT
324027             MIGORI
268698            MOMBASA
197248            NAIROBI
292491             NAKURU
296173              NANDI
275801              NAROK
272704            NYAMIRA
322956          NYANDARUA
259886              NYERI
321531            SAMBURU
318864              SIAYA
15018        TAITA TAVETA
326875         TANA RIVER
331685      THARAKA-NITHI
7571          TRANS NZOIA
25543       

In [473]:
grid_connected = raw_mtf_data_new[raw_mtf_data_new.grid_connection==1].copy()
not_grid_connected =raw_mtf_data_new[raw_mtf_data_new.grid_connection==0].copy()

not_grid_connected.loc[not_grid_connected.generator_for_electricity==2,'generator_for_electricity'] = 0



In [474]:
not_grid_means = not_grid_connected.groupby('county')[['distance_to_grid']].mean().reset_index().rename(columns={'distance_to_grid':'avg_distance_to_connect'})
not_grid_sums = not_grid_connected.groupby('county')[['weighted_generator','pw_final','weighted_batteries','weighted_battery_flag','weighted_solar_flag']].sum().reset_index().rename(columns={})
not_grid_sums['not_grid_generator_pct'] = not_grid_sums['weighted_generator']/not_grid_sums.pw_final
not_grid_sums['not_grid_battery_pct'] = not_grid_sums['weighted_battery_flag']/not_grid_sums.pw_final
not_grid_sums['not_grid_solar_pct'] = not_grid_sums['weighted_solar_flag']/not_grid_sums.pw_final
not_grid_data =pd.merge(not_grid_sums, not_grid_means, on='county',how='outer').drop(columns=['weighted_generator','pw_final','weighted_batteries','weighted_battery_flag','weighted_solar_flag'])

In [475]:
grid_sums = grid_connected.groupby('county')[['weighted_generator','pw_final','weighted_batteries','weighted_battery_flag','weighted_solar_flag']].sum().reset_index().rename(columns={})
grid_sums['grid_generator_pct'] = grid_sums['weighted_generator']/grid_sums.pw_final
grid_sums['grid_battery_pct'] = grid_sums['weighted_battery_flag']/grid_sums.pw_final
grid_sums['grid_solar_pct'] = grid_sums['weighted_solar_flag']/grid_sums.pw_final
grid_not_grid_data =pd.merge(grid_sums,not_grid_data, on='county',how='outer').drop(columns=['weighted_battery_flag','weighted_solar_flag','weighted_generator','pw_final','weighted_batteries'])

In [478]:
summed_dist_data=raw_mtf_data_new.groupby('county')[['pw_final', 'weightedhousehold_inhabitants','weighted_hh_per_dwelling','weighted_owned_dwelling','weighted_rooms_occupied','weighted_grid_connection','weighted_bank_account','weighted_mobile_money','weighted_generator','weighted_battery_flag','weighted_solar_flag']].sum().reset_index().rename(columns={'pw_final':'num_households','weightedhousehold_inhabitants':'total_inhabitants'})
summed_dist_data['avg_household_size'] = summed_dist_data.total_inhabitants/summed_dist_data.num_households
summed_dist_data['avg_household_per_dwelling'] = summed_dist_data.weighted_hh_per_dwelling/summed_dist_data.num_households
summed_dist_data['pct_own_dwelling'] = summed_dist_data.weighted_owned_dwelling/summed_dist_data.num_households
summed_dist_data['avg_rooms_occupied'] = summed_dist_data.weighted_rooms_occupied/summed_dist_data.num_households
summed_dist_data['percent_grid_connect'] = summed_dist_data.weighted_grid_connection/summed_dist_data.num_households
summed_dist_data['percent_bank_account'] = summed_dist_data.weighted_bank_account/summed_dist_data.num_households
summed_dist_data['percent_mobile_money'] = summed_dist_data.weighted_mobile_money/summed_dist_data.num_households
summed_dist_data['total_percent_batteries'] = summed_dist_data.weighted_battery_flag/summed_dist_data.num_households
summed_dist_data['total_percent_solar'] = summed_dist_data.weighted_solar_flag/summed_dist_data.num_households
summed_dist_data['total_percent_generator'] = summed_dist_data.weighted_generator/summed_dist_data.num_households

final_data=summed_dist_data.drop(columns=['weighted_grid_connection','weighted_battery_flag','weighted_generator','weighted_solar_flag','weighted_rooms_occupied','weighted_owned_dwelling','weighted_hh_per_dwelling','weighted_bank_account','weighted_mobile_money'])

mft_data_aggregated=pd.merge(final_data, grid_not_grid_data, on ='county',how='left')
mft_data_aggregated.to_csv("./Datasets/MFT_KENYA_SUMMARY.csv")

In [477]:
mft_data_aggregated

,county,num_households,total_inhabitants,avg_household_size,avg_household_per_dwelling,pct_own_dwelling,avg_rooms_occupied,percent_grid_connect,percent_bank_account,percent_mobile_money,total_percent_batteries,total_percent_solar,total_percent_generator,grid_generator_pct,grid_battery_pct,grid_solar_pct,not_grid_generator_pct,not_grid_battery_pct,not_grid_solar_pct,avg_distance_to_connect
0,BARINGO,5.616115e+06,2.964955e+07,5.279370,1.408945,0.894769,2.437085,0.196069,0.730082,0.922149,0.116308,0.576585,0.019100,0.000000,0.000000,0.083221,0.023759,0.144675,0.696910,1.705446
1,BOMET,2.280848e+06,9.798672e+06,4.296065,1.000000,1.000000,1.841122,0.019881,0.163182,0.867459,0.015406,0.668819,0.000000,0.000000,0.000000,0.000000,0.000000,0.015719,0.682385,2.604167
2,BUNGOMA,2.725576e+07,1.452069e+08,5.327569,2.145785,0.918440,2.873619,0.143707,0.326173,0.847515,0.058666,0.315666,0.000000,0.000000,0.000000,0.192905,0.000000,0.068512,0.336268,15.212027
3,BUSIA,2.747917e+05,1.177679e+06,4.285714,1.000000,1.000000,2.428571,0.000000,0.428571,0.857143,0.000000,0.428571,0.000000,NaN,NaN,NaN,0.000000,0.000000,0.428571,2.000000
4,ELGEYO-MARAKWET,4.328991e+06,2.376124e+07,5.488863,1.000000,0.897477,2.307992,0.103758,0.432059,0.905180,0.000000,0.694102,0.001352,0.013030,0.000000,0.348043,0.000000,0.000000,0.734165,40.033684
5,EMBU,4.482752e+06,1.933036e+07,4.312165,1.475833,0.949228,4.070102,0.128745,0.673514,0.923041,0.095585,0.513875,0.000000,0.000000,0.000000,0.000000,0.000000,0.109710,0.589810,43.772795
6,GARISSA,4.304235e+06,1.468136e+07,3.410912,1.670464,0.688183,2.605738,0.517912,0.240226,0.459516,0.000000,0.123660,0.000000,0.000000,0.000000,0.191098,0.000000,0.000000,0.051210,785.840000
7,HOMA BAY,5.947365e+06,2.267963e+07,3.813391,1.526735,0.879496,2.936897,0.179655,0.302738,0.703372,0.071669,0.310706,0.035517,0.000000,0.000000,0.000000,0.043295,0.087365,0.378750,194.930842
8,ISIOLO,3.104821e+06,1.171439e+07,3.772969,1.115836,0.874923,3.017705,0.291038,0.290352,0.653470,0.000396,0.087296,0.000000,0.000000,0.000000,0.000749,0.000000,0.000558,0.122824,194.867081
9,KAKAMEGA,4.754284e+06,1.993405e+07,4.192861,1.394692,0.812013,2.770939,0.216781,0.222149,0.817166,0.059204,0.279470,0.000000,0.000000,0.000000,0.156060,0.000000,0.075590,0.313628,219.828169


In [334]:
combined_data = pd.merge(population_data, mft_data_aggregated, left_on='county',right_on='dist')

In [337]:
list(combined_data.county)

['Baringo',
 'Bomet',
 'Bungoma North',
 'Bungoma South',
 'Bungoma West',
 'Butere',
 'Embu',
 'Emuhaya',
 'Githunguri',
 'Gucha',
 'Gucha South',
 'Hamisi',
 'Imenti North',
 'Kajiado North',
 'Kakamega Central',
 'Kakamega North',
 'Kaloleni',
 'Kangundo',
 'Kericho',
 'Kikuyu',
 'Kilifi',
 'Kinango',
 'Kipkelion',
 'Kirinyaga',
 'Kisii Central',
 'Kisii South',
 'Kisumu East',
 'Kisumu West',
 'Kitui',
 'Koibatek',
 'Kwale',
 'Kwanza',
 'Lagdera',
 'Laikipia East',
 'Laikipia North',
 'Laikipia West',
 'Lamu',
 'Limuru',
 'Loitokitok',
 'Lugari',
 'Maara',
 'Malindi',
 'Mandera Central',
 'Mandera East',
 'Marsabit',
 'Meru Central',
 'Migori',
 'Molo',
 'Mombasa',
 'Moyale',
 'Msambweni',
 'Mt Elgon',
 'Mutomo',
 'Naivasha',
 'Nakuru',
 'Nakuru North',
 'Nandi Central',
 'Nandi East',
 'Nandi North',
 'Narok North',
 'Narok South',
 'Nyamira',
 'Nyandarua North',
 'Nyandarua South',
 'Nyando',
 'Nyeri South',
 'Nzaui',
 'Pokot Central',
 'Pokot North',
 'Rarieda',
 'Rongo',
 'Ruir

1160

In [354]:
match_div2 = pd.merge(raw_mtf_data[['prov','cty','dist','div','loc']], level2_boundary_lookup, left_on='dist', right_on='ADM2_EN', how = 'left')
match_div1 = pd.merge(raw_mtf_data[['prov','cty','dist','div','loc']], level1_info, left_on='dist', right_on='ADM1_EN', how = 'left')

In [355]:
match_div1

,prov,cty,dist,div,loc,ADM1_EN
0,North Eastern,21.0,Garissa,Central,Central,Garissa
1,Central,6.0,Thika West,Thika West,Makongeni,NaN
2,Rift Valley,33.0,Trans Nzoia West,Kiminini,Kiminini,NaN
3,Rift Valley,43.0,Bomet,Sigor,Sigor,Bomet
4,Rift Valley,30.0,Turkana South,Katilu,Katilu,NaN
...,...,...,...,...,...,...
4581,Coast,9.0,Kilifi,Kikambala,Junju,Kilifi
4582,Western,44.0,Lugari,Likuyani,Nzoia,NaN
4583,Coast,8.0,Msambweni,Lunga Lunga,Lunga Lunga,NaN
4584,Eastern,20.0,Nzaui,Mulala,Emali,NaN


In [362]:

match_div2.loc[match_div2.dist.isin(list(level1_info)),'ADM1_EN']=match_div2.loc[match_div2.dist.isin(list(level1_info)),'dist']
match_div2

,prov,cty,dist,div,loc,ADM1_EN,ADM2_EN
0,North Eastern,21.0,Garissa,Central,Central,Garissa,NaN
1,Central,6.0,Thika West,Thika West,Makongeni,NaN,NaN
2,Rift Valley,33.0,Trans Nzoia West,Kiminini,Kiminini,NaN,NaN
3,Rift Valley,43.0,Bomet,Sigor,Sigor,Bomet,NaN
4,Rift Valley,30.0,Turkana South,Katilu,Katilu,Turkana,Turkana South
...,...,...,...,...,...,...,...
4581,Coast,9.0,Kilifi,Kikambala,Junju,Kilifi,NaN
4582,Western,44.0,Lugari,Likuyani,Nzoia,Kakamega,Lugari
4583,Coast,8.0,Msambweni,Lunga Lunga,Lunga Lunga,Kwale,Msambweni
4584,Eastern,20.0,Nzaui,Mulala,Emali,NaN,NaN


In [368]:
match_div2[pd.isna(match_div2.ADM1_EN)]

,prov,cty,dist,div,loc,ADM1_EN,ADM2_EN
1,Central,6.0,Thika West,Thika West,Makongeni,NaN,NaN
2,Rift Valley,33.0,Trans Nzoia West,Kiminini,Kiminini,NaN,NaN
6,Rift Valley,34.0,Koibatek,Mumberes,Mumberes,NaN,NaN
7,Eastern,18.0,Mwingi,Migwani,Nzauni,NaN,NaN
8,Coast,12.0,Taita,Tausa,Mbololo,NaN,NaN
...,...,...,...,...,...,...,...
4577,North Eastern,23.0,Mandera Central,Rhamu,Rhamu,NaN,NaN
4579,Rift Valley,32.0,Samburu Central,Kirisia,Maralal Town,NaN,NaN
4580,Rift Valley,36.0,Keiyo,Kamariny,Irong,NaN,NaN
4584,Eastern,20.0,Nzaui,Mulala,Emali,NaN,NaN


In [367]:
len(match_div2[pd.isna(match_div2.ADM1_EN)])

1873

In [365]:
level1_info.sort()
level1_info

['Baringo',
 'Bomet',
 'Bungoma',
 'Busia',
 'Elgeyo-Marakwet',
 'Embu',
 'Garissa',
 'Homa Bay',
 'Isiolo',
 'Kajiado',
 'Kakamega',
 'Kericho',
 'Kiambu',
 'Kilifi',
 'Kirinyaga',
 'Kisii',
 'Kisumu',
 'Kitui',
 'Kwale',
 'Laikipia',
 'Lamu',
 'Machakos',
 'Makueni',
 'Mandera',
 'Marsabit',
 'Meru',
 'Migori',
 'Mombasa',
 "Murang'a",
 'Nairobi',
 'Nakuru',
 'Nandi',
 'Narok',
 'Nyamira',
 'Nyandarua',
 'Nyeri',
 'Samburu',
 'Siaya',
 'Taita Taveta',
 'Tana River',
 'Tharaka-Nithi',
 'Trans Nzoia',
 'Turkana',
 'Uasin Gishu',
 'Vihiga',
 'Wajir',
 'West Pokot']